In [26]:
pip install Groq

In [27]:
# Pokemon Arena Experiment - Complete Implementation with Detailed Output
import os
import json
import random
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass, field
from enum import Enum
import time
from groq import Groq
import re
from datetime import datetime, timedelta
import threading
import getpass

In [28]:
# ==================== Data Classes ====================

@dataclass
class Move:
    name: str
    type: str
    category: str  # Physical, Special, Status
    power: int
    accuracy: int
    pp: int
    effect: str = ""
    priority: int = 0  # For priority moves like Quick Attack

@dataclass
class Pokemon:
    name: str
    types: List[str]
    hp: int
    attack: int
    defense: int
    sp_attack: int
    sp_defense: int
    speed: int
    moves: List[Move]
    current_hp: int = None

    def __post_init__(self):
        if self.current_hp is None:
            self.current_hp = self.hp

class TerrainType(Enum):
    WATER = "Water"
    FIRE = "Fire"
    ELECTRIC = "Electric"
    GRASS = "Grass"
    PSYCHIC = "Psychic"
    ROCK = "Rock"
    NORMAL = "Normal"

@dataclass
class BattleState:
    agent_a_team: List[Pokemon]
    agent_b_team: List[Pokemon]
    agent_a_active: int = 0
    agent_b_active: int = 0
    terrain: TerrainType = TerrainType.NORMAL
    turn: int = 0
    agent_a_knocked_out: int = 0
    agent_b_knocked_out: int = 0
    battle_log: List[str] = field(default_factory=list)

In [29]:
# ==================== Pokemon Data Fetcher ====================

class PokemonDataFetcher:
    """Fetches real Pokemon data from PokeAPI and other sources"""

    def __init__(self):
        self.base_url = "https://pokeapi.co/api/v2/"
        self.pokemon_cache = {}
        self.type_chart = self._load_type_effectiveness()

    def _load_type_effectiveness(self) -> Dict[str, Dict[str, float]]:
        """Load type effectiveness chart"""
        type_chart = {
            "Normal": {"Rock": 0.5, "Ghost": 0, "Steel": 0.5},
            "Fire": {"Fire": 0.5, "Water": 0.5, "Grass": 2, "Ice": 2, "Bug": 2, "Rock": 0.5, "Dragon": 0.5, "Steel": 2},
            "Water": {"Fire": 2, "Water": 0.5, "Grass": 0.5, "Ground": 2, "Rock": 2, "Dragon": 0.5},
            "Electric": {"Water": 2, "Electric": 0.5, "Grass": 0.5, "Ground": 0, "Flying": 2, "Dragon": 0.5},
            "Grass": {"Fire": 0.5, "Water": 2, "Grass": 0.5, "Poison": 0.5, "Ground": 2, "Flying": 0.5, "Bug": 0.5, "Rock": 2, "Dragon": 0.5, "Steel": 0.5},
            "Ice": {"Fire": 0.5, "Water": 0.5, "Grass": 2, "Ice": 0.5, "Ground": 2, "Flying": 2, "Dragon": 2, "Steel": 0.5},
            "Fighting": {"Normal": 2, "Ice": 2, "Poison": 0.5, "Flying": 0.5, "Psychic": 0.5, "Bug": 0.5, "Rock": 2, "Ghost": 0, "Dark": 2, "Steel": 2, "Fairy": 0.5},
            "Poison": {"Grass": 2, "Poison": 0.5, "Ground": 0.5, "Rock": 0.5, "Ghost": 0.5, "Steel": 0, "Fairy": 2},
            "Ground": {"Fire": 2, "Electric": 2, "Grass": 0.5, "Poison": 2, "Flying": 0, "Bug": 0.5, "Rock": 2, "Steel": 2},
            "Flying": {"Electric": 0.5, "Grass": 2, "Fighting": 2, "Bug": 2, "Rock": 0.5, "Steel": 0.5},
            "Psychic": {"Fighting": 2, "Poison": 2, "Psychic": 0.5, "Dark": 0, "Steel": 0.5},
            "Bug": {"Fire": 0.5, "Grass": 2, "Fighting": 0.5, "Poison": 0.5, "Flying": 0.5, "Psychic": 2, "Ghost": 0.5, "Dark": 2, "Steel": 0.5, "Fairy": 0.5},
            "Rock": {"Fire": 2, "Ice": 2, "Fighting": 0.5, "Ground": 0.5, "Flying": 2, "Bug": 2, "Steel": 0.5},
            "Ghost": {"Normal": 0, "Psychic": 2, "Ghost": 2, "Dark": 0.5},
            "Dragon": {"Dragon": 2, "Steel": 0.5, "Fairy": 0},
            "Dark": {"Fighting": 0.5, "Psychic": 2, "Ghost": 2, "Dark": 0.5, "Fairy": 0.5},
            "Steel": {"Fire": 0.5, "Water": 0.5, "Electric": 0.5, "Ice": 2, "Rock": 2, "Steel": 0.5, "Fairy": 2},
            "Fairy": {"Fire": 0.5, "Fighting": 2, "Poison": 0.5, "Dragon": 2, "Dark": 2, "Steel": 0.5}
        }
        return type_chart

    def fetch_pokemon(self, pokemon_id: int) -> Optional[Pokemon]:
        """Fetch Pokemon data from PokeAPI"""
        if pokemon_id in self.pokemon_cache:
            return self.pokemon_cache[pokemon_id]

        try:
            response = requests.get(f"{self.base_url}pokemon/{pokemon_id}")
            if response.status_code != 200:
                return None

            data = response.json()
            name = data['name'].capitalize()
            types = [t['type']['name'].capitalize() for t in data['types']]
            stats = {stat['stat']['name']: stat['base_stat'] for stat in data['stats']}

            # Fetch moves with priority
            moves = []
            priority_moves = ['quick-attack', 'mach-punch', 'aqua-jet', 'bullet-punch']
            move_names = ['tackle', 'ember', 'water-gun', 'thunder-shock', 'vine-whip',
                         'ice-beam', 'earthquake', 'psychic', 'bite', 'wing-attack',
                         'rock-throw', 'shadow-ball', 'dragon-claw', 'dark-pulse',
                         'iron-tail', 'moonblast', 'growl', 'tail-whip', 'leer',
                         'thunder-wave', 'toxic', 'protect', 'rest', 'substitute',
                         'quick-attack', 'thunderbolt', 'dragon-rage']

            available_moves = [m['move']['name'] for m in data['moves']]
            selected_moves = []

            for move_name in move_names:
                if move_name in available_moves and len(selected_moves) < 4:
                    move_data = self._fetch_move(move_name)
                    if move_data:
                        selected_moves.append(move_data)

            while len(selected_moves) < 4:
                if len(selected_moves) == 0:
                    selected_moves.append(Move("Tackle", "Normal", "Physical", 40, 100, 35, "", 0))
                elif len(selected_moves) == 1:
                    selected_moves.append(Move("Growl", "Normal", "Status", 0, 100, 40, "Lowers opponent's attack", 0))
                elif len(selected_moves) == 2:
                    selected_moves.append(Move("Quick Attack", "Normal", "Physical", 40, 100, 30, "Priority move", 1))
                else:
                    selected_moves.append(Move("Leer", "Normal", "Status", 0, 100, 30, "Lowers opponent's defense", 0))

            pokemon = Pokemon(
                name=name,
                types=types,
                hp=stats.get('hp', 50),
                attack=stats.get('attack', 50),
                defense=stats.get('defense', 50),
                sp_attack=stats.get('special-attack', 50),
                sp_defense=stats.get('special-defense', 50),
                speed=stats.get('speed', 50),
                moves=selected_moves
            )

            self.pokemon_cache[pokemon_id] = pokemon
            return pokemon

        except Exception as e:
            print(f"Error fetching Pokemon {pokemon_id}: {e}")
            return None

    def _fetch_move(self, move_name: str) -> Optional[Move]:
        """Fetch move data from PokeAPI"""
        try:
            response = requests.get(f"{self.base_url}move/{move_name}")
            if response.status_code != 200:
                return None

            data = response.json()

            # Determine priority
            priority = 1 if move_name in ['quick-attack', 'mach-punch', 'aqua-jet'] else 0

            return Move(
                name=data['name'].replace('-', ' ').title(),
                type=data['type']['name'].capitalize(),
                category=data['damage_class']['name'].capitalize(),
                power=data['power'] or 0,
                accuracy=data['accuracy'] or 100,
                pp=data['pp'] or 10,
                effect=data['effect_entries'][0]['short_effect'] if data['effect_entries'] else "",
                priority=priority
            )
        except:
            return None

    def get_non_legendary_pokemon(self, count: int = 50) -> List[Pokemon]:
        """Get a list of non-legendary Pokemon"""
        # Mix of different generation Pokemon for variety
        pokemon_ids = [
            1, 4, 7, 16, 19, 21, 23, 25, 27, 29, 32, 35, 37, 39, 41, 43, 46, 48, 50, 52,
            54, 56, 58, 60, 63, 66, 69, 72, 74, 77, 79, 81, 83, 84, 86, 88, 90, 92, 95, 96,
            98, 100, 102, 104, 106, 107, 108, 109, 111, 113
        ]

        pokemon_list = []
        for pokemon_id in pokemon_ids[:count]:
            pokemon = self.fetch_pokemon(pokemon_id)
            if pokemon:
                pokemon_list.append(pokemon)
                time.sleep(0.1)  # Rate limiting

        return pokemon_list

In [30]:
# ==================== Battle System ====================

class BattleSystem:
    """Handles Pokemon battles with terrain effects and detailed output"""

    def __init__(self, type_chart: Dict[str, Dict[str, float]]):
        self.type_chart = type_chart
        self.terrain_effects = self._init_terrain_effects()

    def _init_terrain_effects(self) -> Dict[TerrainType, Dict]:
        """Initialize terrain effect modifiers"""
        return {
            TerrainType.WATER: {
                "boost_types": ["Water"],
                "weaken_types": ["Fire"],
                "boost_multiplier": 1.5,
                "weaken_multiplier": 0.7,
                "description": "Water moves boosted by 50%, Fire moves weakened by 30%"
            },
            TerrainType.FIRE: {
                "boost_types": ["Fire"],
                "weaken_types": ["Water", "Ice"],
                "boost_multiplier": 1.5,
                "weaken_multiplier": 0.7,
                "description": "Fire moves boosted by 50%, Water/Ice moves weakened by 30%"
            },
            TerrainType.ELECTRIC: {
                "boost_types": ["Electric"],
                "weaken_types": ["Ground"],
                "boost_multiplier": 1.5,
                "weaken_multiplier": 0.5,
                "description": "Electric moves boosted by 50%, Ground moves weakened by 50%"
            },
            TerrainType.GRASS: {
                "boost_types": ["Grass"],
                "weaken_types": ["Fire"],
                "boost_multiplier": 1.5,
                "weaken_multiplier": 0.7,
                "description": "Grass moves boosted by 50%, Fire moves weakened by 30%"
            },
            TerrainType.PSYCHIC: {
                "boost_types": ["Psychic"],
                "weaken_types": ["Dark"],
                "boost_multiplier": 1.5,
                "weaken_multiplier": 0.7,
                "description": "Psychic moves boosted by 50%, Dark moves weakened by 30%"
            },
            TerrainType.ROCK: {
                "boost_types": ["Rock"],
                "weaken_types": ["Water", "Grass"],
                "boost_multiplier": 1.5,
                "weaken_multiplier": 0.7,
                "description": "Rock moves boosted by 50%, Water/Grass moves weakened by 30%"
            },
            TerrainType.NORMAL: {
                "boost_types": [],
                "weaken_types": [],
                "boost_multiplier": 1.0,
                "weaken_multiplier": 1.0,
                "description": "No terrain effects"
            }
        }

    def get_type_effectiveness_string(self, effectiveness: float) -> str:
        """Get string description of type effectiveness"""
        if effectiveness >= 2.0:
            return "super effective"
        elif effectiveness <= 0.5:
            return "not very effective"
        elif effectiveness == 0:
            return "no effect"
        else:
            return "neutral"

    def calculate_damage(self, attacker: Pokemon, defender: Pokemon, move: Move, terrain: TerrainType) -> Tuple[int, str]:
        """Calculate damage dealt by a move and return damage with description"""
        if move.category == "Status":
            return 0, f"{move.name} ({move.type}, status move)"

        # Base damage formula
        level = 50

        if move.category == "Physical":
            attack_stat = attacker.attack
            defense_stat = defender.defense
        else:
            attack_stat = attacker.sp_attack
            defense_stat = defender.sp_defense

        damage = ((2 * level + 10) / 250) * (attack_stat / defense_stat) * move.power + 2

        # Type effectiveness
        effectiveness = 1.0
        for defender_type in defender.types:
            if move.type in self.type_chart and defender_type in self.type_chart[move.type]:
                effectiveness *= self.type_chart[move.type][defender_type]

        # STAB
        if move.type in attacker.types:
            effectiveness *= 1.5

        # Terrain effects
        terrain_modifier = ""
        terrain_data = self.terrain_effects[terrain]
        if move.type in terrain_data["boost_types"]:
            effectiveness *= terrain_data["boost_multiplier"]
            terrain_modifier = ", boosted by terrain"
        elif move.type in terrain_data["weaken_types"]:
            effectiveness *= terrain_data["weaken_multiplier"]
            terrain_modifier = ", weakened by terrain"

        # Random factor
        random_factor = random.uniform(0.85, 1.0)
        final_damage = int(damage * effectiveness * random_factor)

        # Build move description
        type_matchup = self.get_type_effectiveness_string(effectiveness)
        priority_text = "priority move, " if move.priority > 0 else ""
        defender_types_str = "/".join(defender.types) + "-type " if effectiveness != 1.0 else ""

        move_desc = f"{move.name} ({move.type}{terrain_modifier}, {priority_text}{type_matchup}"
        if type_matchup != "neutral":
            move_desc += f" vs. {defender_types_str}{defender.name}"
        move_desc += ")"

        return max(1, final_damage), move_desc

    def execute_turn(self, state: BattleState, agent_a_move: Move, agent_b_move: Move) -> Tuple[List[str], bool]:
        """Execute a turn of battle and return turn descriptions"""
        turn_logs = []
        battle_ended = False

        pokemon_a = state.agent_a_team[state.agent_a_active]
        pokemon_b = state.agent_b_team[state.agent_b_active]

        # Determine turn order
        a_priority = agent_a_move.priority
        b_priority = agent_b_move.priority

        if a_priority > b_priority:
            first = ("A", pokemon_a, agent_a_move, pokemon_b)
            second = ("B", pokemon_b, agent_b_move, pokemon_a)
        elif b_priority > a_priority:
            first = ("B", pokemon_b, agent_b_move, pokemon_a)
            second = ("A", pokemon_a, agent_a_move, pokemon_b)
        elif pokemon_a.speed > pokemon_b.speed:
            first = ("A", pokemon_a, agent_a_move, pokemon_b)
            second = ("B", pokemon_b, agent_b_move, pokemon_a)
        elif pokemon_b.speed > pokemon_a.speed:
            first = ("B", pokemon_b, agent_b_move, pokemon_a)
            second = ("A", pokemon_a, agent_a_move, pokemon_b)
        else:
            if random.random() < 0.5:
                first = ("A", pokemon_a, agent_a_move, pokemon_b)
                second = ("B", pokemon_b, agent_b_move, pokemon_a)
            else:
                first = ("B", pokemon_b, agent_b_move, pokemon_a)
                second = ("A", pokemon_a, agent_a_move, pokemon_b)

        state.turn += 1

        # Start turn log
        turn_log = f"Turn {state.turn}:"
        move_logs = []

        # Execute moves
        for agent_id, attacker, move, defender in [first, second]:
            if attacker.current_hp <= 0:
                continue

            # Check accuracy
            if random.randint(1, 100) > move.accuracy:
                move_logs.append(f"{attacker.name} uses {move.name} (missed).")
                continue

            # Calculate and apply damage
            damage, move_desc = self.calculate_damage(attacker, defender, move, state.terrain)
            defender.current_hp = max(0, defender.current_hp - damage)

            # Build move log
            move_log = f"{attacker.name} uses {move_desc}."
            if damage > 0:
                if defender.current_hp > 0:
                    move_log += f" {defender.name} takes {damage} damage."
                else:
                    move_log += f" {defender.name} takes heavy damage."

            move_logs.append(move_log)

            # Check if defender fainted
            if defender.current_hp <= 0:
                move_logs.append(f"{defender.name} faints.")

                if agent_id == "A":
                    state.agent_b_knocked_out += 1
                else:
                    state.agent_a_knocked_out += 1

                # Check for next Pokemon
                if agent_id == "A":
                    available = [p for p in state.agent_b_team if p.current_hp > 0]
                    if available:
                        # Find next available Pokemon
                        for i, p in enumerate(state.agent_b_team):
                            if p.current_hp > 0:
                                state.agent_b_active = i
                                move_logs.append(f"Agent B sends {p.name}.")
                                break
                    else:
                        battle_ended = True
                else:
                    available = [p for p in state.agent_a_team if p.current_hp > 0]
                    if available:
                        for i, p in enumerate(state.agent_a_team):
                            if p.current_hp > 0:
                                state.agent_a_active = i
                                move_logs.append(f"Agent A sends {p.name}.")
                                break
                    else:
                        battle_ended = True
                break

        # Combine turn log
        turn_log = f"Turn {state.turn}: " + " ".join(move_logs)
        turn_logs.append(turn_log)

        return turn_logs, battle_ended

In [31]:
# ==================== LLM Agent ====================

class RateLimiter:
    """Rate limiter for API calls"""
    def __init__(self, calls_per_minute=20, calls_per_day=400):
        self.calls_per_minute = calls_per_minute
        self.calls_per_day = calls_per_day
        self.minute_window = []
        self.day_window = []
        self.lock = threading.Lock()

    def can_make_call(self):
        with self.lock:
            now = datetime.now()
            minute_ago = now - timedelta(minutes=1)
            day_ago = now - timedelta(days=1)

            self.minute_window = [t for t in self.minute_window if t > minute_ago]
            self.day_window = [t for t in self.day_window if t > day_ago]

            if len(self.minute_window) >= self.calls_per_minute:
                return False
            if len(self.day_window) >= self.calls_per_day:
                return False

            return True

    def wait_if_needed(self):
        while not self.can_make_call():
            time.sleep(2)

    def record_call(self):
        with self.lock:
            now = datetime.now()
            self.minute_window.append(now)
            self.day_window.append(now)

class LLMAgent:
    """LLM-based Pokemon trainer agent"""

    rate_limiter = RateLimiter(calls_per_minute=15, calls_per_day=300)

    def __init__(self, agent_id: str, groq_api_key: str, model: str = "llama-3.3-70b-versatile"):
        self.agent_id = agent_id
        self.groq_client = Groq(api_key=groq_api_key)
        self.model = model
        self.strategy_profile = self._generate_strategy_profile()
        self.preferred_types = self._generate_type_preferences()

    def _generate_strategy_profile(self) -> str:
        """Generate a unique strategy profile for the agent"""
        profiles = [
            "Aggressive attacker focusing on high-damage moves and type advantages",
            "Defensive strategist prioritizing Pokemon longevity and status moves",
            "Balanced trainer adapting strategies based on opponent patterns",
            "Terrain specialist maximizing environmental advantages",
            "Speed-focused trainer leveraging fast Pokemon and priority moves",
            "Tank builder using high HP and defense Pokemon",
            "Type coverage expert ensuring diverse team composition",
            "Risk-taker favoring high-risk, high-reward strategies"
        ]
        return random.choice(profiles)

    def _generate_type_preferences(self) -> List[str]:
        """Generate preferred Pokemon types for this agent"""
        all_types = ["Fire", "Water", "Grass", "Electric", "Psychic", "Ice",
                     "Dragon", "Dark", "Fighting", "Flying", "Poison", "Ground",
                     "Rock", "Bug", "Ghost", "Steel", "Normal", "Fairy"]
        # Each agent prefers 3-5 types
        num_preferences = random.randint(3, 5)
        return random.sample(all_types, num_preferences)

    def select_roster(self, available_pokemon: List[Pokemon], roster_size: int = 10) -> List[Pokemon]:
        """Select initial roster with agent-specific preferences"""
        # Score each Pokemon based on agent preferences
        scored_pokemon = []

        for pokemon in available_pokemon:
            score = 0

            # Base score on total stats
            total_stats = pokemon.hp + pokemon.attack + pokemon.defense + pokemon.sp_attack + pokemon.sp_defense + pokemon.speed
            score += total_stats / 100

            # Bonus for preferred types
            for ptype in pokemon.types:
                if ptype in self.preferred_types:
                    score += 30

            # Strategy-specific bonuses
            if "Aggressive" in self.strategy_profile:
                score += (pokemon.attack + pokemon.sp_attack) / 50
            elif "Defensive" in self.strategy_profile:
                score += (pokemon.defense + pokemon.sp_defense + pokemon.hp) / 75
            elif "Speed" in self.strategy_profile:
                score += pokemon.speed / 25
            elif "Tank" in self.strategy_profile:
                score += (pokemon.hp * 2 + pokemon.defense + pokemon.sp_defense) / 100

            # Add randomness to ensure variety
            score += random.uniform(-10, 10)

            scored_pokemon.append((score, pokemon))

        # Sort by score and select top Pokemon
        scored_pokemon.sort(key=lambda x: x[0], reverse=True)
        selected = [p[1] for p in scored_pokemon[:roster_size]]

        return selected

    def select_battle_team(self, roster: List[Pokemon], terrain: TerrainType, team_size: int = 6) -> List[Pokemon]:
        """Select battle team based on terrain"""
        terrain_advantages = {
            TerrainType.WATER: ["Water", "Electric"],
            TerrainType.FIRE: ["Fire", "Rock", "Ground"],
            TerrainType.ELECTRIC: ["Electric", "Flying"],
            TerrainType.GRASS: ["Grass", "Bug", "Poison"],
            TerrainType.PSYCHIC: ["Psychic", "Fairy"],
            TerrainType.ROCK: ["Rock", "Ground", "Steel"],
            TerrainType.NORMAL: self.preferred_types  # Use agent preferences
        }

        preferred_for_terrain = terrain_advantages.get(terrain, [])

        # Score each Pokemon
        scored_pokemon = []
        for pokemon in roster:
            score = 0

            # Base score
            total_stats = pokemon.hp + pokemon.attack + pokemon.defense + pokemon.sp_attack + pokemon.sp_defense + pokemon.speed
            score += total_stats / 100

            # Terrain bonus
            for ptype in pokemon.types:
                if ptype in preferred_for_terrain:
                    score += 40

            # Move type bonus
            for move in pokemon.moves:
                if move.type in preferred_for_terrain:
                    score += 15

            # Add some randomness
            score += random.uniform(-5, 5)

            scored_pokemon.append((score, pokemon))

        # Sort and select
        scored_pokemon.sort(key=lambda x: x[0], reverse=True)
        team = [self._copy_pokemon(p[1]) for p in scored_pokemon[:team_size]]

        return team

    def _copy_pokemon(self, pokemon: Pokemon) -> Pokemon:
        """Create a fresh copy of a Pokemon"""
        return Pokemon(
            name=pokemon.name,
            types=pokemon.types,
            hp=pokemon.hp,
            attack=pokemon.attack,
            defense=pokemon.defense,
            sp_attack=pokemon.sp_attack,
            sp_defense=pokemon.sp_defense,
            speed=pokemon.speed,
            moves=pokemon.moves
        )

    def choose_move(self, state: BattleState, is_agent_a: bool) -> Move:
        """Choose a move based on strategy"""
        if is_agent_a:
            my_pokemon = state.agent_a_team[state.agent_a_active]
            opp_pokemon = state.agent_b_team[state.agent_b_active]
        else:
            my_pokemon = state.agent_b_team[state.agent_b_active]
            opp_pokemon = state.agent_a_team[state.agent_a_active]

        # Score each move
        move_scores = []

        for move in my_pokemon.moves:
            score = 0

            if move.category != "Status":
                # Base score on power
                score += move.power

                # Type effectiveness
                effectiveness = 1.0
                for opp_type in opp_pokemon.types:
                    if move.type in self._get_simple_type_chart() and opp_type in self._get_simple_type_chart()[move.type]:
                        effectiveness *= self._get_simple_type_chart()[move.type][opp_type]

                score *= effectiveness

                # STAB bonus
                if move.type in my_pokemon.types:
                    score *= 1.5

                # Priority move bonus when low on health
                if move.priority > 0 and my_pokemon.current_hp < my_pokemon.hp * 0.3:
                    score *= 2

                # Strategy modifiers
                if "Aggressive" in self.strategy_profile:
                    score *= 1.2
                elif "Defensive" in self.strategy_profile and move.category == "Status":
                    score *= 1.5
            else:
                # Status moves
                if "Defensive" in self.strategy_profile:
                    score = 50
                else:
                    score = 20

            # Add randomness
            score += random.uniform(-10, 10)
            move_scores.append((score, move))

        # Select best move
        move_scores.sort(key=lambda x: x[0], reverse=True)
        return move_scores[0][1]

    def _get_simple_type_chart(self):
        """Simplified type chart for quick calculations"""
        return {
            "Fire": {"Water": 0.5, "Grass": 2, "Ice": 2, "Bug": 2, "Steel": 2},
            "Water": {"Fire": 2, "Ground": 2, "Rock": 2},
            "Grass": {"Water": 2, "Ground": 2, "Rock": 2},
            "Electric": {"Water": 2, "Flying": 2},
            "Ice": {"Grass": 2, "Ground": 2, "Flying": 2, "Dragon": 2},
            "Fighting": {"Normal": 2, "Rock": 2, "Steel": 2, "Ice": 2, "Dark": 2},
            "Poison": {"Grass": 2, "Fairy": 2},
            "Ground": {"Fire": 2, "Electric": 2, "Poison": 2, "Rock": 2, "Steel": 2},
            "Flying": {"Fighting": 2, "Bug": 2, "Grass": 2},
            "Psychic": {"Fighting": 2, "Poison": 2},
            "Bug": {"Grass": 2, "Psychic": 2, "Dark": 2},
            "Rock": {"Fire": 2, "Ice": 2, "Flying": 2, "Bug": 2},
            "Ghost": {"Psychic": 2, "Ghost": 2},
            "Dragon": {"Dragon": 2},
            "Dark": {"Psychic": 2, "Ghost": 2},
            "Steel": {"Ice": 2, "Rock": 2, "Fairy": 2},
            "Fairy": {"Fighting": 2, "Dragon": 2, "Dark": 2}
        }

In [32]:
# ==================== Tournament System ====================

class Tournament:
    """Manages the Pokemon tournament with detailed output"""

    def __init__(self, agents: List[LLMAgent], battle_system: BattleSystem):
        self.agents = agents
        self.battle_system = battle_system
        self.results = []
        self.scores = {agent.agent_id: 0 for agent in agents}

    def run_battle(self, agent_a: LLMAgent, agent_b: LLMAgent,
                   team_a: List[Pokemon], team_b: List[Pokemon],
                   terrain: TerrainType, battle_num: int) -> Dict:
        """Run a single battle with detailed output"""
        state = BattleState(
            agent_a_team=team_a,
            agent_b_team=team_b,
            terrain=terrain
        )

        print(f"\nBattle {battle_num} ({terrain.value} Terrain):")

        # Initial setup
        print(f"Turn 1: Agent {agent_a.agent_id} leads with {team_a[0].name}; "
              f"Agent {agent_b.agent_id} chooses {team_b[0].name}.")

        max_turns = 50
        battle_logs = []

        while state.turn < max_turns:
            # Check for knocked out Pokemon
            if state.agent_a_team[state.agent_a_active].current_hp <= 0:
                available_a = [(i, p) for i, p in enumerate(state.agent_a_team) if p.current_hp > 0]
                if available_a:
                    state.agent_a_active = available_a[0][0]
                    print(f"Agent {agent_a.agent_id} sends {state.agent_a_team[state.agent_a_active].name}.")
                else:
                    break

            if state.agent_b_team[state.agent_b_active].current_hp <= 0:
                available_b = [(i, p) for i, p in enumerate(state.agent_b_team) if p.current_hp > 0]
                if available_b:
                    state.agent_b_active = available_b[0][0]
                    print(f"Agent {agent_b.agent_id} sends {state.agent_b_team[state.agent_b_active].name}.")
                else:
                    break

            # Get moves
            move_a = agent_a.choose_move(state, True)
            move_b = agent_b.choose_move(state, False)

            # Execute turn
            turn_logs, battle_ended = self.battle_system.execute_turn(state, move_a, move_b)

            for log in turn_logs:
                print(log)
                battle_logs.append(log)

            if battle_ended:
                break

        # Determine winner
        a_remaining = sum(1 for p in state.agent_a_team if p.current_hp > 0)
        b_remaining = sum(1 for p in state.agent_b_team if p.current_hp > 0)

        if b_remaining == 0:
            winner = agent_a.agent_id
            winner_remaining = a_remaining
            loser_remaining = 0
        else:
            winner = agent_b.agent_id
            winner_remaining = b_remaining
            loser_remaining = a_remaining

        print(f"Battle continues with Agent {winner} winning {winner_remaining}-{loser_remaining}, "
              f"leveraging {terrain.value} terrain and type advantages.")

        return {
            "winner": winner,
            "agent_a": agent_a.agent_id,
            "agent_b": agent_b.agent_id,
            "winner_knockouts": state.agent_b_knocked_out if winner == agent_a.agent_id else state.agent_a_knocked_out,
            "loser_knockouts": state.agent_a_knocked_out if winner == agent_a.agent_id else state.agent_b_knocked_out,
            "turns": state.turn,
            "terrain": terrain.value,
            "battle_logs": battle_logs
        }

    def run_match(self, agent_a: LLMAgent, agent_b: LLMAgent,
                  roster_a: List[Pokemon], roster_b: List[Pokemon],
                  match_type: str = "Semifinal", best_of: int = 5) -> Dict:
        """Run a best-of-N match with detailed output"""
        print(f"\n{match_type} Match: Agent {agent_a.agent_id} vs. Agent {agent_b.agent_id}")

        # Display rosters
        print(f"Roster (Agent {agent_a.agent_id}): {', '.join([p.name for p in roster_a])}.")
        print(f"Roster (Agent {agent_b.agent_id}): {', '.join([p.name for p in roster_b])}.")

        match_results = []
        wins_a = 0
        wins_b = 0

        for battle_num in range(1, best_of + 1):
            terrain = random.choice(list(TerrainType))

            # Select teams
            team_a = agent_a.select_battle_team(roster_a, terrain, team_size=3)
            team_b = agent_b.select_battle_team(roster_b, terrain, team_size=3)

            # Run battle
            result = self.run_battle(agent_a, agent_b, team_a, team_b, terrain, battle_num)
            match_results.append(result)

            if result["winner"] == agent_a.agent_id:
                wins_a += 1
            else:
                wins_b += 1

            # Check if match decided
            if wins_a > best_of // 2 or wins_b > best_of // 2:
                break

            time.sleep(1)

        # Calculate points
        match_winner = agent_a.agent_id if wins_a > wins_b else agent_b.agent_id
        total_knockouts = sum(r["winner_knockouts"] if r["winner"] == match_winner else r["loser_knockouts"]
                             for r in match_results)

        # Award points
        win_points = 10
        knockout_points = total_knockouts * 2
        strategic_points = 4 if total_knockouts > len(match_results) * 2 else 2
        adaptability_points = 2

        total_points = win_points + knockout_points + strategic_points + adaptability_points

        print(f"\nSeries Outcome: Agent {match_winner} wins the series {wins_a if match_winner == agent_a.agent_id else wins_b}-"
              f"{wins_b if match_winner == agent_a.agent_id else wins_a}, earning {win_points} points (win), "
              f"{knockout_points} points ({total_knockouts} knockouts), {strategic_points} points (strategic efficiency), "
              f"and {adaptability_points} points (adaptability).")

        if match_type == "Semifinal":
            print(f"Agent {match_winner} advances to the final.")

        # Update scores
        self.scores[match_winner] += total_points

        return {
            "match_winner": match_winner,
            "wins_a": wins_a,
            "wins_b": wins_b,
            "total_points": total_points,
            "battles": match_results
        }

In [33]:
# ==================== Main Execution ====================

def main():
    """Main execution function"""

    # Configuration
    GROQ_API_KEY = getpass.getpass("Enter your Groq API key: ")
    NUM_AGENTS = 2
    NUM_POKEMON = 30
    ROSTER_SIZE = 6
    TEAM_SIZE = 3
    BEST_OF = 5

    print("\nInitializing Pokemon Arena Tournament...")

    # Initialize systems
    print("Loading Pokemon data...")
    data_fetcher = PokemonDataFetcher()
    available_pokemon = data_fetcher.get_non_legendary_pokemon(NUM_POKEMON)
    print(f"Loaded {len(available_pokemon)} Pokemon")

    battle_system = BattleSystem(data_fetcher.type_chart)

    # Create agents
    print("\nCreating AI agents...")
    agents = []
    for i in range(NUM_AGENTS):
        agent = LLMAgent(f"{chr(65 + i)}", GROQ_API_KEY)  # A, B, C, D...
        print(f"Agent {agent.agent_id}: {agent.strategy_profile}")
        print(f"  Preferred types: {', '.join(agent.preferred_types)}")
        agents.append(agent)
        time.sleep(1)

    # Tournament
    print("\n" + "="*60)
    print("POKEMON ARENA TOURNAMENT")
    print("="*60)

    # Roster selection
    rosters = {}
    for agent in agents:
        roster = agent.select_roster(available_pokemon, ROSTER_SIZE)
        rosters[agent.agent_id] = roster

    # Run matches
    tournament = Tournament(agents, battle_system)

    # For 2 agents, just one match (can expand for more agents)
    if NUM_AGENTS == 2:
        match_result = tournament.run_match(
            agents[0], agents[1],
            rosters[agents[0].agent_id], rosters[agents[1].agent_id],
            "Final", BEST_OF
        )

    # Display final results
    print("\n" + "="*60)
    print("TOURNAMENT RESULTS")
    print("="*60)

    sorted_agents = sorted(tournament.scores.items(), key=lambda x: x[1], reverse=True)
    for rank, (agent_id, score) in enumerate(sorted_agents, 1):
        print(f"{rank}. Agent {agent_id}: {score} points")

    if sorted_agents:
        print(f"\nTOURNAMENT CHAMPION: Agent {sorted_agents[0][0]}")

    return tournament

if __name__ == "__main__":
    tournament = main()

Enter your Groq API key: ··········

Initializing Pokemon Arena Tournament...
Loading Pokemon data...
Loaded 30 Pokemon

Creating AI agents...
Agent A: Terrain specialist maximizing environmental advantages
  Preferred types: Normal, Fighting, Electric, Water, Bug
Agent B: Aggressive attacker focusing on high-damage moves and type advantages
  Preferred types: Bug, Rock, Ice

POKEMON ARENA TOURNAMENT

Final Match: Agent A vs. Agent B
Roster (Agent A): Pikachu, Tentacool, Poliwag, Spearow, Rattata, Mankey.
Roster (Agent B): Venonat, Paras, Geodude, Ekans, Clefairy, Oddish.

Battle 1 (Electric Terrain):
Turn 1: Agent A leads with Pikachu; Agent B chooses Venonat.
Turn 1: Pikachu uses Iron Tail (Steel, neutral). Venonat takes 44 damage. Venonat uses Psychic (Psychic, neutral). Pikachu takes 32 damage.
Turn 2: Pikachu uses Iron Tail (Steel, neutral). Venonat takes heavy damage. Venonat faints. Agent B sends Clefairy.
Turn 3: Pikachu uses Iron Tail (Steel, super effective vs. Fairy-type Cle